# Clasificación de imágenes pulmonares de rayos X mediante redes neuronales convolucionales

Autores:


*   Alicia Portela Estévez
*   Álvaro García López
*   Álvaro Marínez Petit


## Introducción

La neumonía es una infección que puede afectar a uno a ambos pulmones, causada tanto por virus como por hongos o bacterias. Según la OMS, en 2019 el número de fallecidos a causa de esta enfermedad fue de más de 2 millones de personas en todo el mundo, por lo que un diágnóstico temprano ayudaría a disminuir este número [[Ref.]](https://www.who.int/es/news-room/fact-sheets/detail/the-top-10-causes-of-death)

En el trabajo final de la asignatura "Ingeniería de Grandes Volúmenes de Datos" pretendemos crear un modelo de "Deep Learning" basado en Redes Neuronales Convolucionales que sea capaz de distinguir entre imágenes pulmonares de rayos X procedentes de pacientes con y sin neumonía. De esta forma, podríamos desarrollar una herramienta informática que automatice con cierta calidad el diagnóstico temprano de estos casos.

Dada la gran cantidad de datos que manejamos, hemos utilizado una solución basada en Apache Spark para llevar a cabo el entrenamiento de la red neuronal. De esta forma, el proceso se realiza de forma óptima.

## Preparación del entorno de trabajo

Instalar JAVA (jdk8)

In [ ]:
# Instalamos jdk8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
# Establecemos la ruta del entorno jdk para poder ejecutar Pyspark en Colab
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java


update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


Instalar Analytics Zoo

In [ ]:
# Instalamos la última versión de Analytics Zoo
# Mediante esta acción se instalan automáticamente pyspark, bigdl y sus dependencias.
!pip install --pre --upgrade analytics-zoo[ray]
exit()

     |████████████████████████████████| 194.7 MB 64 kB/s 
     |████████████████████████████████| 114.0 MB 29 kB/s 
     |████████████████████████████████| 218.4 MB 59 kB/s 
     |████████████████████████████████| 65 kB 4.4 MB/s 
     |████████████████████████████████| 47.5 MB 66.0 MB/s 
     |████████████████████████████████| 1.3 MB 50.9 MB/s 
     |████████████████████████████████| 62 kB 817 kB/s 
     |████████████████████████████████| 271 kB 61.4 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 197 kB 55.7 MB/s 
     |████████████████████████████████| 202 kB 57.0 MB/s 
     |████████████████████████████████| 173 kB 59.1 MB/s 
     |████████████████████████████████| 82 kB 265 kB/s 
     |████████████████████████████████| 3.0 MB 45.1 MB/s 
     |████████████████████████████████| 128 kB 45.6 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218

Importamos Orca

In [ ]:
# Importamos las librerías y módulos necesarios
from zoo.orca import init_orca_context, stop_orca_context
from zoo.orca import OrcaContext

# Fijando este parámetro a 'True' se muestra en el terminal del Jupyter Notebook el stdout y el stderr
OrcaContext.log_output = True

# Iniciamos el contexto Orca
init_orca_context(cluster_mode="local", cores=1)

# Obtenemos la sesión de spark iniciada por Orca
spark = OrcaContext.get_spark_session()

Initializing orca context
Current pyspark location is : /usr/local/lib/python3.7/dist-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.7/dist-packages/zoo/share/lib/analytics-zoo-bigdl_0.13.1-SNAPSHOT-spark_2.4.6-0.12.0-SNAPSHOT-jar-with-dependencies.jar:/usr/local/lib/python3.7/dist-packages/bigdl/share/lib/bigdl-0.13.1-SNAPSHOT-jar-with-dependencies.jar pyspark-shell 
2022-02-10 11:26:07 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).

User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   K

cls.getname: com.intel.analytics.bigdl.python.api.Sample
BigDLBasePickler registering: bigdl.util.common  Sample
cls.getname: com.intel.analytics.bigdl.python.api.EvaluatedResult
BigDLBasePickler registering: bigdl.util.common  EvaluatedResult
cls.getname: com.intel.analytics.bigdl.python.api.JTensor
BigDLBasePickler registering: bigdl.util.common  JTensor
cls.getname: com.intel.analytics.bigdl.python.api.JActivity
BigDLBasePickler registering: bigdl.util.common  JActivity
Successfully got a SparkContext


Importamos otras librerías y módulos necesarios

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Comprobamos la versión instalada de TensorFlow y Keras

In [ ]:
tf.__version__

'2.7.0'

In [ ]:
keras.__version__

'2.7.0'

## Importación de los datos y creación del modelo

Definimos tres funciones que serán llamadas posteriormente en el modelo, cuya finalidad es cargar los datos.

Cada una de ellas se encarga de cargar uno de los subconjuntos utilizados en el desarrollo del modelo (entrenamiento, validación y evaluación).

In [ ]:
# Ruta al directorio base que contiene todas las imágenes de rayos X a utilizar
basepath = "/content/drive/MyDrive/MBC/Big_Data_Engineering/Big_Data_Final_Project/chest_xray"

# Función para cargar los datos de entrenamiento
def train_data_creator(config, batch_size):
    # Creamos una instancia de ImageDataGenerator que reescala los colores 
    # (en formato RGB) de los píxeles de 0 a 1 en lugar de la codificación 
    # original de 0 a 255, y voltea algunas imágenes horizontalmente al azar
    # para aumentar la diversidad de los datos y que el modelo no pierda
    # precisión por la orientación de la persona a la hora de tomar la radiografía.
    # Esto último solo es necesario en los datos de entrenamiento, por lo que
    # no lo repetimos en los datos de validación y evaluación
    datagen_train = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

    # Usamos la instancia de ImageDataGenerator para cargar los datos del directorio
    # especificado con el tamaño, modo de color y tamaño de 'batch' requerido
    training_set = datagen_train.flow_from_directory(os.path.join(basepath, "train"),
                                              target_size=(150, 150),
                                              color_mode="grayscale",
                                              batch_size=batch_size)

    return training_set

# Función para cargar los datos de validación
def val_data_creator(config, batch_size):
    # Creamos una instancia de ImageDataGenerator que reescala los colores 
    # (en formato RGB) de los píxeles de 0 a 1 en lugar de la codificación 
    # original de 0 a 255.
    datagen_val = ImageDataGenerator(rescale=1./255)

    # Usamos la instancia de ImageDataGenerator para cargar los datos del directorio
    # especificado con el tamaño, modo de color y tamaño de 'batch' requerido
    val_set = datagen_val.flow_from_directory(os.path.join(basepath, "val"),
                                          target_size=(150, 150),
                                          color_mode="grayscale",
                                          batch_size=batch_size)
    return val_set

# Función para cargar los datos de evaluación
def test_data_creator(config, batch_size):
    # Creamos una instancia de ImageDataGenerator que reescala los colores 
    # (en formato RGB) de los píxeles de 0 a 1 en lugar de la codificación 
    # original de 0 a 255.
    datagen_test = ImageDataGenerator(rescale=1./255)

    # Usamos la instancia de ImageDataGenerator para cargar los datos del directorio
    # especificado con el tamaño, modo de color y tamaño de 'batch' requerido
    test_set = datagen_test.flow_from_directory(os.path.join(basepath, "test"),
                                          target_size=(150, 150),
                                          color_mode="grayscale",
                                          batch_size=batch_size)
    return test_set

Definimos el modelo que vamos a utilizar

In [ ]:
# Función en la que se define el modelo de red neuronal
def model_creator(config):
    # Utilizamos el modo Secuencial de keras para la construcción de la red
    model = tf.keras.Sequential()

    # Creamos 5 capas de 32 neuronas convolucionales, definiendo la función de
    # activación como 'relu' y el tamaño de los datos de entrada en la primera de todas
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), input_shape=(150, 150, 1), activation="relu"))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
    # Creamos una capa de MaxPooling para reducir la dimensionalidad de la imagen y
    # así reducir el coste computacional y minimizar la posibilidad de 'overfitting'
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    # Creamos una capa Flatten para convertir la salida bidimensional de la capa
    # anterior en una entrada unidimensional de la siguiente
    model.add(tf.keras.layers.Flatten())
    # Creamos una capa de 10 neuronas completamente conectadas
    model.add(tf.keras.layers.Dense(10, activation="relu"))
    # Creamos una capa de 2 neuronas como salida, con función de activación 'softmax'
    # para realizar una clasificación binaria
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    # Compilamos el modelo con 'adam' como optimizador, 'categorical_crossentropy'
    # como función de pérdida y obteniendo métricas de 'accuracy', 'precision', 
    # 'recall' y 'area under the curve' (AUC)
    model.compile(optimizer="Adam", 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy",
                            keras.metrics.Precision(name='precision'), 
                            keras.metrics.Recall(name='recall'), 
                            keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
                           ])
    
    return model

Creamos una instancia del 'Estimator' proporcionado por Orca (Analytics Zoo) a partir del modelo previamente definido

In [ ]:
from zoo.orca.learn.tf2 import Estimator

est = Estimator.from_keras(model_creator=model_creator, workers_per_node=1)

(pid=5224) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/zoo/orca/learn/tf2/tf_runner.py:317: _CollectiveAllReduceStrategyExperimental.__init__ (from tensorflow.python.distribute.collective_all_reduce_strategy) is deprecated and will be removed in a future version.
(pid=5224) Instructions for updating:
(pid=5224) use distribute.MultiWorkerMirroredStrategy instead
(pid=5224) 2022-02-10 12:30:48.294105: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## Entrenamiento del modelo

Definimos 3 épocas y 32 como tamaño de 'batch' para entrenar el Estimador, llamando a las funciones que cargan los datos

In [ ]:
max_epoch = 3
batch_size = 32

stats = est.fit(train_data_creator,
                epochs=max_epoch,
                batch_size=batch_size,
                validation_data=val_data_creator)


(pid=5224) Found 5043 images belonging to 2 classes.
(pid=5224) Found 10 images belonging to 2 classes.


(pid=5224) 2022-02-10 12:30:52.195401: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
(pid=5224) op: "FlatMapDataset"
(pid=5224) input: "TensorDataset/_1"
(pid=5224) attr {
(pid=5224)   key: "Targuments"
(pid=5224)   value {
(pid=5224)     list {
(pid=5224)     }
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "_cardinality"
(pid=5224)   value {
(pid=5224)     i: -2
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "f"
(pid=5224)   value {
(pid=5224)     func {
(pid=5224)       name: "__inference_Dataset_flat_map_flat_map_fn_372"
(pid=5224)     }
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "metadata"
(pid=5224)   value {
(pid=5224)     s: "\n\020FlatMapDataset:1"
(pid=5224)   }
(pid=5224) }
(pid=5

(pid=5224) Epoch 1/3
 39/158 [======>.......................] - ETA: 12:09 - loss: 0.5559 - accuracy: 0.7263 - precision: 0.7263 - recall: 0.7263 - prc: 0.7750
(pid=5224) 
158/158 [==============================] - ETA: 0s - loss: 0.3835 - accuracy: 0.8245 - precision: 0.8245 - recall: 0.8245 - prc: 0.9116


(pid=5224) 2022-02-10 12:47:11.720466: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
(pid=5224) op: "FlatMapDataset"
(pid=5224) input: "TensorDataset/_1"
(pid=5224) attr {
(pid=5224)   key: "Targuments"
(pid=5224)   value {
(pid=5224)     list {
(pid=5224)     }
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "_cardinality"
(pid=5224)   value {
(pid=5224)     i: -2
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "f"
(pid=5224)   value {
(pid=5224)     func {
(pid=5224)       name: "__inference_Dataset_flat_map_flat_map_fn_3521"
(pid=5224)     }
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "metadata"
(pid=5224)   value {
(pid=5224)     s: "\n\021FlatMapDataset:21"
(pid=5224)   }
(pid=5224) }
(pid

158/158 [==============================] - 982s 6s/step - loss: 0.3835 - accuracy: 0.8245 - precision: 0.8245 - recall: 0.8245 - prc: 0.9116 - val_loss: 0.2573 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_prc: 1.0000
(pid=5224) Epoch 2/3
158/158 [==============================] - ETA: 0s - loss: 0.2587 - accuracy: 0.9284 - precision: 0.9284 - recall: 0.9284 - prc: 0.9711


(pid=5224) 2022-02-10 13:03:09.542188: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


158/158 [==============================] - 953s 6s/step - loss: 0.2587 - accuracy: 0.9284 - precision: 0.9284 - recall: 0.9284 - prc: 0.9711 - val_loss: 0.2499 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_prc: 1.0000
(pid=5224) Epoch 3/3
108/158 [===================>..........]
(pid=5224)  - ETA: 5:01 - loss: 0.2038 - accuracy: 0.9547 - precision: 0.9547 - recall: 0.9547 - prc: 0.9829
152/158 [===========================>..]
(pid=5224)  - ETA: 36s - loss: 0.1963 - accuracy: 0.9561 - precision: 0.9561 - recall: 0.9561 - prc: 0.9841
158/158 [==============================] - ETA: 0s - loss: 0.1971 - accuracy: 0.9560 - precision: 0.9560 - recall: 0.9560 - prc: 0.9839


(pid=5224) 2022-02-10 13:19:33.039456: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


158/158 [==============================] - 955s 6s/step - loss: 0.1971 - accuracy: 0.9560 - precision: 0.9560 - recall: 0.9560 - prc: 0.9839 - val_loss: 0.1590 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_prc: 1.0000


## Evaluación del modelo

Una vez el Estimador ha entrenado correctamente, lo evaluamos cargando los datos de evaluación llamando a su función correspondiente

In [ ]:
stats = est.evaluate(test_data_creator, 
                     batch_size = batch_size)

print(stats)

(pid=5224) Found 586 images belonging to 2 classes.


(pid=5224) 2022-02-10 13:35:32.274057: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
(pid=5224) op: "FlatMapDataset"
(pid=5224) input: "TensorDataset/_1"
(pid=5224) attr {
(pid=5224)   key: "Targuments"
(pid=5224)   value {
(pid=5224)     list {
(pid=5224)     }
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "_cardinality"
(pid=5224)   value {
(pid=5224)     i: -2
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "f"
(pid=5224)   value {
(pid=5224)     func {
(pid=5224)       name: "__inference_Dataset_flat_map_flat_map_fn_7861"
(pid=5224)     }
(pid=5224)   }
(pid=5224) }
(pid=5224) attr {
(pid=5224)   key: "metadata"
(pid=5224)   value {
(pid=5224)     s: "\n\021FlatMapDataset:95"
(pid=5224)   }
(pid=5224) }
(pid

19/19 [==============================] - 26s 1s/step - loss: 0.7189 - accuracy: 0.7799 - precision: 0.7799 - recall: 0.7799 - prc: 0.8432
{'validation_loss': 0.7188912630081177, 'validation_accuracy': 0.7798634767532349, 'validation_precision': 0.7798634767532349, 'validation_recall': 0.7798634767532349, 'validation_prc': 0.8432269096374512}


Mejoramos la visualización de las métricas obtenidas tras la evaluación

In [ ]:
for stat, value in stats.items():
  print (stat, "\t", value, sep = "")

validation_loss	0.7188912630081177
validation_accuracy	0.7798634767532349
validation_precision	0.7798634767532349
validation_recall	0.7798634767532349
validation_prc	0.8432269096374512


Conseguimos finalmente un 'accuracy' del 78%, lo que nos da una visión global de la calidad de la clasificación.

De forma más específica, los resultados de 'precision' y 'recall' nos muestran que se producen el mismo número de errores en la clasificación de falsos negativos como de falsos positivos.

Por último, el resultado de 'Precision-Recall curve' es de 0'84, un valor superior al umbral de 0'5 para considerar válido el modelo.

Consideramos que este resultado es suficientemente alto para utilizar el modelo como una primera aproximación al diagnóstico de neumonía, pero siempre con la supervisión de un profesional.



## Guardado del modelo

Guardamos el modelo entrenado para poder usarlo en el futuro

In [ ]:
model_path = "/content/drive/MyDrive/MBC/Big_Data_Engineering/Big_Data_Final_Project/"
est.save(os.path.join(model_path, "big_data_final_model_1"))
# est.shutdown()

'/content/drive/MyDrive/MBC/Big_Data_Engineering/Big_Data_Final_Project/big_data_final_model_1'

## Comentarios finales

Como hemos visto a lo largo del código, hemos utilizado datos en grandes cantidades para generar un modelo que pueda predecir con relativamente alta precisión (78%) si una radiografía corresponde a un paciente con neumonía o no.

Con la idea de explorar más opciones, nos planteamos realizar más transformaciones a las imágenes de entrenamiento, como se muestra en el código a continuación, con el fin de introducir más variabilidad en el modelo. Sin embargo, las métricas de los resultados (precisión del 72%) no mejoraban las del modelo final expuesto. Además, consideramos que tampoco se genera tanta variabilidad en el entorno en el que se consiguen las imágenes, ya que la toma de radiografías de pecho es un proceso bastante estandarizado y no suele tener variaciones de brillo o ángulo.

In [ ]:
#datagen_train = ImageDataGenerator(rescale=1./255, 
#                                   horizontal_flip=True,
#                                   rotation_range=15,
#                                   brightness_range=[0.5,1.0],
#                                   zoom_range=[0.9,1.0])